In [90]:
import pandas as pd
import numpy as np
import pylab as plt
import os

###### read ML data ######
dfMLOrigin = pd.read_excel('data/ML_data/descriptor_4717MOF.xlsx')

###### read flexibility data ##### 
flexibilityList=os.listdir('data/flexibility_data/y_data/adsorption_data')
flexivilityData=[]

for i, name in enumerate(flexibilityList):   
    df=pd.read_csv('data/flexibility_data/y_data/adsorption_data/'+name)
    rigidValue=np.array(df[df.columns[1]],dtype=float)
    flexValue=np.mean(np.array(df[df.columns[2:]],dtype=float),axis=1)
    label=np.array([name[24:28] for x in range(0,len(flexValue))],dtype=str) 
    singleSet=np.column_stack([rigidValue,flexValue,label])

    if i==0:
        MOFName=np.array(df[df.columns[0]],dtype='U6')
        flexivilityData=singleSet.copy()
    else:
        flexivilityData=np.concatenate([flexivilityData.copy(),singleSet.copy()])
        
##### match two dataSet #######

dfML= dfMLOrigin[dfMLOrigin['MOF'].isin(MOFName)]

print(dfML.head(2))
print(flexivilityData[0:3,:])


    ID     MOF Periodic Chemical Formula  ρ(g.cm-3)  PLD (Å)  LCD (Å)  \
12  13  ABUWOJ                H7C12O7Zn2    1.15833  4.03039  5.07969   
24  25  ACOLIP              H19C22N5O4Zn    1.04949  3.57647  4.91034   

    VSA  (m2/cm3)  GSA (m2/g)        vf  vp (cm3/g)  ...  F01[Ne-Rh]  \
12        1007.55     869.832  0.545974    0.532253  ...           0   
24           0.00       0.000  0.454051    0.521040  ...           0   

    F01[Ne-Pd]  F01[Ne-Ag]  F01[Ne-Cd]  F01[Ne-In]  F01[Ne-Sn]  F01[Ne-Sb]  \
12           0           0           0           0           0           0   
24           0           0           0           0           0           0   

    F01[Ne-Te]  F01[Ne-I]  F01[Ne-Xe]  
12           0          0           0  
24           0          0           0  

[2 rows x 1024 columns]
[['5.395625473' '5.016580520775' 'xeno']
 ['5.788896266' '4.817801024044944' 'xeno']
 ['2.323461698' '3.3505466442417773' 'xeno']]
